In [2]:
!pip install pandas openai



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install xlrd

In [2]:
from openai import OpenAI, types

client = OpenAI() 
client.api_key ="sk-Z2OiBLrrPNVXvl7EaHZj8SHMjXOo0FoOSHkkSlcwrLT3BlbkFJL3qaWbssfsw0iMYCXs-dCdjdwIo0wbQ3t2-WcWD0IA"
# Your code below:
response = client.chat.completions.create( 
  model="gpt-3.5-turbo", 
  messages=[ 
    { 
      "role": "user", 
      "content": "Tell me how many seconds there are in a day in a Shakespearean tone?" 
    } 
  ]) 

print(response.choices[0].message.content) 

There are eight and twenty thousand moments in the span of a sun-kissed day, as fleeting as shadows in the dusk, each heartbeat a second on the grand stage of time.


In [5]:
print(response.choices[0].message.content) 

Forsooth, in a single day doth pass
Twenty-four hours, a thousand moments fast
Yet in each hour, sixty seconds doth remain
Thus in a day, doth four and twenty times sixty reign
A total of fourteen hundred and forty, be sure
Seconds in a day, ever passing, ever pure.


In [6]:
# Import the necessary libraries and initialize the client
from openai import OpenAI, types

client = OpenAI()
client.api_key ="sk-Z2OiBLrrPNVXvl7EaHZj8SHMjXOo0FoOSHkkSlcwrLT3BlbkFJL3qaWbssfsw0iMYCXs-dCdjdwIo0wbQ3t2-WcWD0IA"

# Sample paper details for testing
paper_details = {
    "title": "Mobility Patterns and the Spread of Infectious Diseases: Insights from Public Datasets",
    "abstract": (
        "This study leverages publicly available mobility data to analyze the effects of human movement "
        "patterns on the spread of infectious diseases. Using large-scale datasets derived from urban transit "
        "and cellphone location data, we explore correlations between mobility trends and infection rates. "
        "Findings suggest that reduced movement is associated with slower disease transmission in major metropolitan areas."
    ),
    "public_availability": "Yes",  # Can be "Yes" or "No" depending on paper's access policy
    "dataset_info": "The datasets used are from public transit authorities and anonymized cellphone data aggregates."
}

# Construct the prompt with clear instructions.
prompt = f"""
You are an expert in epidemiology and data science. I need you to evaluate the following research paper based on the criteria below:

1. **Relevance**: On a scale from 0 to 100, how strongly is this paper focused on the topic of human movement analysis and its effect on infectious diseases?
2. **Public Data Usage**: Does the paper use publicly available mobility datasets? Answer with "Yes" or "No".
3. **Justification**: Provide a brief explanation of your evaluation.

Here is the paper information:

Title: {paper_details['title']}

Abstract: {paper_details['abstract']}

Public Availability: {paper_details['public_availability']}

Dataset Information: {paper_details['dataset_info']}

Please provide your response in the following JSON format:

{{
  "relevance_score": <number between 0 and 100>,
  "public_data": "<Yes or No>",
  "justification": "<your explanation>"
}}
"""

# Call the OpenAI chat completion API
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

# Print the output from the model
print(response.choices[0].message.content)


ImportError: cannot import name 'OpenAI' from 'openai' (c:\ProgramData\anaconda3\Lib\site-packages\openai\__init__.py)

In [7]:
import pandas as pd
import json
import time
from openai import OpenAI, types

# Initialize the OpenAI client with your API key
client = OpenAI()
client.api_key ="sk-Z2OiBLrrPNVXvl7EaHZj8SHMjXOo0FoOSHkkSlcwrLT3BlbkFJL3qaWbssfsw0iMYCXs-dCdjdwIo0wbQ3t2-WcWD0IA"

# Read the Excel file into a pandas DataFrame.
# Adjust the file path if necessary.
df = pd.read_excel("savedrecs.xls")

# Create an empty list to store evaluation results
evaluation_results = []

# Define a function that builds the prompt for a given paper (row)
def build_prompt(row):
    # Extract fields from the row
    # Adjust the column names if necessary to match your Excel file
    title = row.get("Article Title", "No Title Provided")
    abstract = row.get("Abstract", "No Abstract Provided")
    # Optional: You may include other details if available. For example:
    # source_title = row.get("Source Title", "No Source Provided")
    # For this example, we focus on the title and abstract.
    
    prompt = f"""
You are an expert in epidemiology and data science. I need you to evaluate the following research paper based on the criteria below:

1. **Relevance**: On a scale from 0 to 100, how strongly is this paper focused on the topic of human movement analysis and its effect on infectious diseases?
2. **Public Data Usage**: Does the paper use publicly available mobility datasets? Answer with "Yes" or "No".
3. **Justification**: Provide a brief explanation of your evaluation.

Here is the paper information:

Title: {title}

Abstract: {abstract}

Please provide your response in the following JSON format:

{{
  "relevance_score": <number between 0 and 100>,
  "public_data": "<Yes or No>",
  "justification": "<your explanation>"
}}
    """
    return prompt

# Iterate over each paper in the DataFrame
# For demonstration purposes, you might want to process only a subset of rows.
for index, row in df.iterrows():
    prompt_text = build_prompt(row)
    
    try:
        # Call the OpenAI Chat Completion API
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt_text}
            ]
        )
        
        # Get the message content from the response
        response_content = response.choices[0].message.content.strip()
        print(f"Response for record {index}:")
        print(response_content)
        
        # Attempt to parse the response as JSON
        try:
            response_data = json.loads(response_content)
        except json.JSONDecodeError:
            # If the response is not valid JSON, store the raw string
            response_data = {"error": "Invalid JSON response", "raw_response": response_content}
        
        # Append the result along with a reference to the original paper (for example, title)
        evaluation_results.append({
            "index": index,
            "title": row.get("Article Title", "No Title Provided"),
            "evaluation": response_data
        })
    
    except Exception as e:
        print(f"An error occurred processing record {index}: {e}")
        evaluation_results.append({
            "index": index,
            "title": row.get("Article Title", "No Title Provided"),
            "evaluation": {"error": str(e)}
        })
    
    # Optional: Pause for a short duration to respect API rate limits
    time.sleep(1)

# Convert the results into a DataFrame for further analysis or saving
results_df = pd.DataFrame(evaluation_results)

# Display the evaluation results
print(results_df)

# Optionally, you can save the results to a new Excel or CSV file:
results_df.to_excel("evaluation_results.xlsx", index=False)
# or
results_df.to_csv("evaluation_results.csv", index=False)


ImportError: cannot import name 'OpenAI' from 'openai' (c:\ProgramData\anaconda3\Lib\site-packages\openai\__init__.py)

In [4]:
from openai import OpenAI
client = OpenAI()
client.api_key ="sk-Z2OiBLrrPNVXvl7EaHZj8SHMjXOo0FoOSHkkSlcwrLT3BlbkFJL3qaWbssfsw0iMYCXs-dCdjdwIo0wbQ3t2-WcWD0IA"

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Code calls itself back,  \nLayers unfold endlessly—  \nLoops within the loop.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [5]:
import random
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI()
client.api_key = "sk-Z2OiBLrrPNVXvl7EaHZj8SHMjXOo0FoOSHkkSlcwrLT3BlbkFJL3qaWbssfsw0iMYCXs-dCdjdwIo0wbQ3t2-WcWD0IA"

# List of random abstracts (you can replace these with actual abstracts)
abstracts = [
    "This study explores the relationship between urban planning and pedestrian behavior in metropolitan areas.",
    "The effects of global warming on marine ecosystems have been studied extensively over the past decade.",
    "Human mobility patterns during pandemics reveal significant shifts in travel behavior and work locations.",
    "Advancements in renewable energy have led to a significant reduction in carbon emissions globally.",
    "Analyzing migration trends over the past century highlights the socio-economic factors driving relocation."
]

# Choose a random abstract
random_abstract = random.choice(abstracts)

# Make a completion request to the OpenAI API
response = client.chat.completions.create(
    model="gpt-4o",  # You can replace this with the desired GPT model
    messages=[
        {"role": "system", "content": "You are an expert on human mobility and its related topics."},
        {"role": "user", "content": f"Rate the relevance of the following abstract to the topic of human mobility on a scale of 1 to 10. Provide a brief explanation for your score.\n\nAbstract: {random_abstract}"}
    ]
)

# Extract the model's response
relevance_score = response.choices[0].message.content

# Print the abstract and the relevance score
print(f"Abstract: {random_abstract}\n")
print(f"Relevance Score and Explanation: {relevance_score}")


Abstract: This study explores the relationship between urban planning and pedestrian behavior in metropolitan areas.

Relevance Score and Explanation: I would rate the relevance of this abstract to the topic of human mobility as an 8 out of 10. 

Explanation: Human mobility refers to the movement of people across various spaces, and pedestrian behavior is a fundamental aspect of this. The abstract discusses the connection between urban planning and how pedestrians move and behave in metropolitan areas, which directly relates to understanding and influencing human mobility patterns. Urban planning can significantly impact the efficiency, safety, and accessibility of pedestrian pathways, all of which are crucial elements of human mobility in urban contexts. Therefore, the study's focus on these interactions makes it highly relevant to the topic.


In [6]:
pip install openxyl

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement openxyl (from versions: none)
ERROR: No matching distribution found for openxyl

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import openai
import pandas as pd

# Initialize OpenAI API
client.api_key = "sk-Z2OiBLrrPNVXvl7EaHZj8SHMjXOo0FoOSHkkSlcwrLT3BlbkFJL3qaWbssfsw0iMYCXs-dCdjdwIo0wbQ3t2-WcWD0IA"

# Read Excel file
file_path = "savedrecs.xls"  # Replace with the actual file path
data = pd.read_excel(file_path)

# Extract relevant columns
relevant_columns = ["Article Title", "Abstract", "DOI"]
papers = data[relevant_columns]

# Function to evaluate relevance
def evaluate_paper(title, abstract):
    prompt = f"""
    You are an expert on human mobility and infectious disease modeling. Evaluate the following research paper for its relevance to:
    1. The analysis of human movement effects on infectious diseases.
    2. The availability of publicly accessible mobility datasets for such studies.

    Provide a score (1 to 10) and an explanation for each criterion. Return the results as a JSON object with "relevance_score", "dataset_score", and "explanation".

    Title: {title}
    Abstract: {abstract}
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an expert in human mobility and infectious diseases."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing paper: {title}. Error: {e}")
        return None

# Process each paper
results = []
for index, row in papers.head(5).iterrows():
    title = row["Article Title"]
    abstract = row["Abstract"]
    doi = row["DOI"]
    
    if pd.notna(title) and pd.notna(abstract):
        print(f"Evaluating paper: {title}")
        result = evaluate_paper(title, abstract)
        if result:
            results.append({
                "Title": title,
                "DOI": doi,
                "Evaluation": result
            })




Evaluating paper: Comparing metapopulation dynamics of infectious diseases under different models of human movement
Evaluating paper: Evaluating the impact of human flow on the spread of infectious diseases
Evaluating paper: Modeling Impact of Temperature and Human Movement on the Persistence of Dengue Disease
Evaluating paper: Human migration and infectious diseases


ModuleNotFoundError: No module named 'openpyxl'

In [9]:
results

[{'Title': 'Comparing metapopulation dynamics of infectious diseases under different models of human movement',
  'DOI': '10.1073/pnas.2007488118',
  'Evaluation': '{\n"relevance_score": 9,\n"dataset_score": 7,\n"explanation": {\n    "relevance_explanation": "The paper is highly relevant to the analysis of human movement effects on infectious diseases as it explicitly discusses various models of human movement and their impact on disease dynamics. The paper evaluates and compares three well-studied models of infectious diseases with two possible movement models and then discusses the effect on the outcomes. The relevance score is a 9.",\n    "dataset_explanation": "The paper mentions how newly available datasets are being used to construct models that help in understanding the spread of diseases across large geographical areas. However, it does not explicitly mention that these datasets are publicly accessible or provide specifics on what the datasets consist of. Thus, the paper provid

In [8]:
import openpyxl
# Save results to a new Excel file
output_file = "paper_relevance_scores.xlsx"
output_data = pd.DataFrame(results)
output_data.to_excel(output_file, index=False)
print(f"Results saved to {output_file}")

ModuleNotFoundError: No module named 'openpyxl'